# Environment

In [14]:
import numpy as np
import json
from datetime import datetime
from sklearn.linear_model import LinearRegression

# Data Loading

In [15]:
data_path = "../scrape/data/data.npy"
dates_path = "../scrape/data/dates.json"
tweets_path = "../scrape/data/tweets.json"

data = np.load(data_path)

with open(dates_path) as json_file:  
    dates = json.load(json_file)
    
with open(tweets_path) as json_file:  
    tweets = json.load(json_file)

# Estimation

Steps:

Z is our latent variable, the stock price change from t-2 to t-1. A is our causal variable, the number of times musk tweeted from t-1 to t. Y is our outcome, the stock price change from t-1 to t.

1. Do linear regression to find E[Y|X=x, S=s].
2. Use our function from #1 to calculate E[Y|do(X=x)] = $\sum_{s}Pr(S=s)E[Y|X=x, S=s] \approx \frac{1}{n}\sum_{i=1}^{n}E[Y|X=x,S=s_{i}]$.

# Regression

## Linear Regression

In [75]:
def linear_regression_model(data):
    model = LinearRegression().fit(data[:,0:data.shape[1]-1], data[:,data.shape[1]-1])
    return model

In [26]:
model = LinearRegression().fit(data[:,0:4], data[:,4])
r_sq = model.score(data[:,0:4], data[:,4])
print('coefficient of determination:', r_sq)
#y_pred = model.predict(data[:,0:4])
#y_pred
#np.mean(np.abs(data[:,4] - y_pred))
#np.mean(np.abs(data[:,4]))
#np.std(np.abs(data[:,4]))

# Causal Estimation

In [44]:
def average_treatment_estimation_backdoor(model, data, do_x_1, do_x_2):
    num_1 = 0
    total_1 = 0
    
    num_2 = 0
    total_2 = 0
    
    for i in range(len(data)):
        if data[i, 3] == do_x_1:
            num_1 = num_1 + 1
            total_1 = total_1 + data[i, 4]
            
        if data[i, 3] == do_x_2:
            num_2 = num_2 + 1
            total_2 = total_2 + data[i, 4]
    
    total_1 = total_1/num_1
    total_2 = total_2/num_2
    
    return total_2 - total_1

In [73]:
do_x_1 = 0
do_x_2 = 2

ATE_backdoor = average_treatment_estimation_backdoor(linear_regression_model(data), data, do_x_1, do_x_2)

In [74]:
ATE_backdoor

0.5802315188659977